In [16]:
import pandas as pd
import random as rnd 
import mysql.connector
from mysql.connector import Error
import plotly
import json
import csv
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pymysql
import plotly.express as px
import plotly.graph_objects as go
from sqlalchemy import create_engine
import networkx as nx
from networkx_viewer import Viewer
from scipy.spatial.distance import pdist, squareform
import igraph
import sklearn
from sklearn.cluster import KMeans

## Get data from SQL servers

In [17]:
connection = mysql.connector.connect(host='localhost',
                                         database='employment',
                                         user='root',
                                         password='1234')

In [18]:
def get_data(connection, descriptor_domain,filt = False,val = None):
    '''
    Return df in the SOCs-decriptor format of a specific domain 
    descriptor_domeain:
        work context
        knowledge
        skills
        gwas
        job_zones, filter
        Abilities, filter
    
    '''
    df= pd.DataFrame(data=None)
    if descriptor_domain == 'work_context':
        df= pd.read_sql('''
            SELECT title, element_name, data_value FROM employment.work_context
            LEFT JOIN employment.content_model_reference USING (element_id) 
            LEFT JOIN employment.occupation_data USING (onetsoc_code)
            WHERE scale_id LIKE 'CX';''', con = connection)
        df = df.reset_index().pivot(index = 'title', columns = 'element_name', values = 'data_value')
    elif descriptor_domain == 'job_zones':
        if filt:
            df= pd.read_sql('''
                SELECT title,job_zone FROM employment.job_zones
                LEFT JOIN employment.occupation_data USING (onetsoc_code); ''', con = connection)
            df = df[df['job_zone'].isin(val)]
        else:
            df= pd.read_sql('''
                SELECT title,job_zone FROM employment.job_zones
                LEFT JOIN employment.occupation_data USING (onetsoc_code);''', con = connection)
    elif descriptor_domain == 'abilities':
        if filt:
            df= pd.read_sql('''
            SELECT title, element_name, data_value FROM employment.''' + descriptor_domain + '''
            LEFT JOIN employment.content_model_reference USING (element_id) 
            LEFT JOIN employment.occupation_data USING (onetsoc_code)
            WHERE element_id LIKE\''''+ val +'''%\'; ''', con = connection)
            
        else:
            df= pd.read_sql('''
            SELECT title, element_name,element_id, data_value FROM employment.''' + descriptor_domain + '''
            LEFT JOIN employment.content_model_reference USING (element_id) 
            LEFT JOIN employment.occupation_data USING (onetsoc_code);''', con = connection)
        df = df.groupby(by=['title','element_name'])['data_value'].mean()
        df = df.reset_index().pivot(index = 'title', columns = 'element_name', values = 'data_value')
    elif descriptor_domain == 'work_values':
        df= pd.read_sql('''
            SELECT title, element_name, data_value FROM employment.work_values
            LEFT JOIN employment.content_model_reference USING (element_id) 
            LEFT JOIN employment.occupation_data USING (onetsoc_code)
            WHERE scale_id LIKE 'EX';''', con = connection)
        df = df.reset_index().pivot(index = 'title', columns = 'element_name', values = 'data_value')
    elif descriptor_domain == 'interests':
        df= pd.read_sql('''
            SELECT title, element_name, data_value FROM employment.interests
            LEFT JOIN employment.content_model_reference USING (element_id) 
            LEFT JOIN employment.occupation_data USING (onetsoc_code)
            WHERE scale_id LIKE 'OI';''', con = connection)
        df = df.reset_index().pivot(index = 'title', columns = 'element_name', values = 'data_value')
    elif descriptor_domain == 'work_styles':
        df= pd.read_sql('''
            SELECT title, element_name, data_value FROM employment.work_styles
            LEFT JOIN employment.content_model_reference USING (element_id) 
            LEFT JOIN employment.occupation_data USING (onetsoc_code)
            WHERE scale_id LIKE 'IM';''', con = connection)
        df = df.reset_index().pivot(index = 'title', columns = 'element_name', values = 'data_value')
    elif descriptor_domain == 'name':
        df= pd.read_sql('''
            SELECT * FROM employment.occupation_data;''', con = connection)
    else:
        df= pd.read_sql('''
            SELECT title, element_name, data_value FROM employment.''' + descriptor_domain + '''
            LEFT JOIN employment.content_model_reference USING (element_id) 
            LEFT JOIN employment.occupation_data USING (onetsoc_code);''', con = connection)
        df = df.groupby(by=['title','element_name'])['data_value'].mean()
        df = df.reset_index().pivot(index = 'title', columns = 'element_name', values = 'data_value')
    return df

## Calculating euclidean distance 

In [19]:
def distance_table(df, jz=False, Z = False):
    '''
    @param df: dataframe of 
    
    '''
#     squareform(pdist(df.iloc[:, 1:])), columns=df.title.unique(), index=df.title.unique()
    dist = []
    name_dict = {}
    if jz:
        name_dict = df['title'].sort_values().reset_index()['title'].to_dict()
        dist = pdist(df[df.columns[1:]],'euclidean') #Euclidean distance for 1XN
    else:
        for i in range(len(df.index)):
            name_dict[i] = df.index[i]
        dist = pdist(df,'euclidean') #Euclidean distance for 1XN
    df_dist = pd.DataFrame(squareform(dist)) #Euclidean distance for NxN
#     return df_dist
    columns = list(df_dist.index) 
    for i in range(len(df.index)): #convert 0 values to NaN
        df_dist.loc[i,i] = np.NaN
    mean = df_dist.mean().mean()
    std = df_dist.stack().std() 
    
    if Z:
        for col in columns: #Z-score calculated 
            df_dist[col] = (df_dist[col] - mean)/std
#     return df_dist
    return df_dist.rename(columns = name_dict, index = name_dict)

In [73]:
df_skills = get_data(connection,'skills')
df_skills[:10]

element_name,Active Learning,Active Listening,Complex Problem Solving,Coordination,Critical Thinking,Equipment Maintenance,Equipment Selection,Installation,Instructing,Judgment and Decision Making,...,Science,Service Orientation,Social Perceptiveness,Speaking,Systems Analysis,Systems Evaluation,Technology Design,Time Management,Troubleshooting,Writing
title,,,,,,,,,,,,,,,,,,,,,
Accountants,3.185,3.940,3.185,3.000,3.815,0.5,0.500,0.500,2.815,3.500,...,1.750,3.000,3.000,3.750,3.435,2.940,1.185,3.060,0.815,3.435
Actors,2.620,3.685,2.750,2.940,3.000,0.5,0.500,0.500,2.685,2.940,...,1.000,2.185,3.750,3.940,1.810,1.810,0.815,3.000,0.500,3.130
Actuaries,3.750,4.060,4.310,3.000,4.500,0.5,0.500,0.500,3.250,4.500,...,2.060,3.000,2.880,3.940,4.190,4.250,1.375,3.310,0.500,3.690
Acupuncturists,3.310,3.685,3.125,2.940,3.815,0.5,0.875,0.500,2.685,3.500,...,2.370,3.500,3.685,3.560,2.685,2.935,1.310,2.685,1.125,3.375
Acute Care Nurses,3.815,3.940,3.750,3.815,4.000,0.5,1.440,0.500,3.690,3.875,...,3.060,3.940,4.310,4.000,3.060,3.125,1.560,3.185,1.810,3.560
Adapted Physical Education Specialists,3.560,4.120,3.120,3.375,3.560,0.5,0.750,0.685,3.810,3.560,...,1.000,3.810,4.065,3.880,3.000,3.185,1.435,3.315,1.565,3.685
Adhesive Bonding Machine Operators and Tenders,2.500,2.935,2.625,3.000,2.815,3.0,2.120,0.620,2.315,2.560,...,1.000,2.125,2.630,2.870,2.685,2.000,0.750,2.940,3.000,2.500
"Administrative Law Judges, Adjudicators, and Hearing Officers",3.875,4.935,4.120,3.060,4.750,0.5,0.500,0.500,2.940,4.370,...,1.190,2.880,3.750,4.560,2.815,3.065,1.000,3.250,0.500,4.560
Administrative Services Managers,3.250,4.000,3.185,3.685,3.815,0.5,0.685,0.500,3.125,3.185,...,0.500,3.250,3.500,3.880,3.120,2.935,1.125,4.000,1.565,3.750


In [26]:
# K-Means Cluster 

In [43]:
kmeans = KMeans(n_clusters = 7,random_state = 5)
kmeans


KMeans(n_clusters=7, random_state=5)

In [63]:
Kmodel = kmeans.fit(df_skills)
# Kmodel.labels_

In [46]:
Kmodel.cluster_centers_

array([[2.39961165, 3.11092233, 2.39936893, 2.72796117, 2.86684466,
        0.68825243, 0.75237864, 0.51747573, 2.0692233 , 2.54640777,
        1.99645631, 1.05878641, 1.18757282, 1.93597087, 1.84718447,
        2.76796117, 2.09805825, 1.73718447, 1.44650485, 1.03276699,
        2.26800971, 0.7411165 , 1.62640777, 2.86728155, 0.62524272,
        0.70606796, 2.76033981, 2.80597087, 3.01174757, 1.75975728,
        1.64980583, 0.82029126, 2.59932039, 1.18645631, 2.53796117],
       [2.91719828, 3.16267241, 3.05767241, 2.9624569 , 3.34521552,
        3.17241379, 2.69077586, 1.95362069, 2.67366379, 3.025     ,
        2.58482759, 1.4437931 , 1.7225    , 2.43448276, 2.41310345,
        3.19581897, 2.24547414, 3.45672414, 3.2299569 , 1.64056034,
        2.41737069, 1.17767241, 3.2462069 , 3.15172414, 3.23387931,
        1.64612069, 2.5587069 , 2.75198276, 3.04068966, 2.59146552,
        2.53133621, 1.57844828, 2.94586207, 3.27724138, 2.81780172],
       [3.78391667, 4.11719444, 3.65822222, 3.

In [69]:
def clusters(df,cluster_list):
    '''
    @param: df: dataframe of occupations
            cluster_list: list of different clusters
    returns: dictionary of cluster and their respective occupations 
    '''
    occ_dict = {}
    for i in range(7):
        occ_dict[i] = set()
    
    for occ,cl in zip(df.reset_index()['title'],cluster_list):
        occ_dict[cl].add(occ)
    return occ_dict  

In [81]:
kmeans = KMeans(n_clusters = 7,random_state = 5)
Kmodel = kmeans.fit(df_skills[:25])

clusters(df_skills[:25],Kmodel.labels_)

{0: {'Accountants', 'Actuaries', 'Advertising and Promotions Managers'},
 1: {'Agricultural Inspectors',
  'Agricultural Technicians',
  'Air Traffic Controllers',
  'Aircraft Cargo Handling Supervisors'},
 2: {'Acupuncturists',
  'Adapted Physical Education Specialists',
  'Administrative Law Judges, Adjudicators, and Hearing Officers',
  'Administrative Services Managers',
  'Adult Basic and Secondary Education and Literacy Teachers and Instructors',
  'Advanced Practice Psychiatric Nurses',
  'Advertising Sales Agents',
  'Agents and Business Managers of Artists, Performers, and Athletes',
  'Agricultural Sciences Teachers, Postsecondary'},
 3: {'Aerospace Engineering and Operations Technicians',
  'Aircraft Mechanics and Service Technicians',
  'Aircraft Structure, Surfaces, Rigging, and Systems Assemblers'},
 4: {'Acute Care Nurses', 'Aerospace Engineers', 'Agricultural Engineers'},
 5: {'Adhesive Bonding Machine Operators and Tenders',
  'Agricultural Equipment Operators'},
 6: {